In [1]:
import numpy as np
import pandas as pd
import datetime as dt
import matplotlib.pyplot as plt

from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()

from ipywidgets import interact, interactive_output, fixed, interact_manual
import ipywidgets as widgets

## Analysis of historical London house prices

First, load the csv of historical house price data. The most recent version of the csv file (Feb 2020) is at:

https://www.gov.uk/government/statistical-data-sets/uk-house-price-index-data-downloads-december-2019

Find the file called "UK HPI full file", download and place it in the same directory as this notebook.

In [2]:
df = pd.read_csv("UK-HPI-full-file-2019-12.csv")

Only care about London data, so look for area codes starting 'E09':

In [3]:
london_regions = df[df.AreaCode.apply(lambda x: x[:3]) == 'E09'].RegionName.unique()
london_df = df[df.RegionName.isin(london_regions)].copy()
london_df.Date = pd.to_datetime(london_df.Date, dayfirst=True)

In [4]:
plt.rcParams['figure.figsize'] = (15, 7)
chosen_boroughs = widgets.SelectMultiple(options=london_regions,
                                         rows=10,
                                         value = ['Barking and Dagenham'],
                                         description='Boroughs')

house_type = widgets.Dropdown(options=['Average', 'Detached', 'Flat', 'SemiDetached', 'Terraced', 
                                       'Cash', 'Mortgage', 'FTB', 'FOO', 'New', 'Old'])

metric = widgets.Dropdown(options=['Price', 'Index', 'SalesVolume', '1m%Change', '12m%Change', 'PriceSA', 'IndexSA'])

vbox = widgets.VBox([house_type, metric])
hbox = widgets.HBox([chosen_boroughs, vbox])

def compare_boroughs(chosen_boroughs=chosen_boroughs,
                     house_type=house_type,
                     metric=metric):
    
    if metric != 'Price' and house_type == 'Average':
        house_type = ''
        
    if house_type + metric not in london_df.columns:
        print('No data for this borough/metric combination!')
        return
    
    for borough in chosen_boroughs:
        this_df = london_df[london_df.RegionName == borough].copy()
        this_df = this_df[['Date', house_type + metric]].set_index('Date')
        plt.plot(this_df, label=borough)
    if metric == 'Index':
        plt.vlines(dt.date(day=1, month=1, year=2015), ymin=plt.ylim()[0], ymax=plt.ylim()[1])    
    plt.legend()
    plt.grid()
    
ui = widgets.HBox([chosen_boroughs, 
                   widgets.HBox([widgets.VBox([widgets.Label('Comparison type'), widgets.Label('Metric')]), 
                                 widgets.VBox([house_type, metric])
                                ])
                  ])
out = interactive_output(compare_boroughs, {'chosen_boroughs': chosen_boroughs,
                                      'house_type':house_type,
                                      'metric':metric})
display(ui, out)

Output()